# 匯入資料集

In [196]:
import pandas as pd
#關鍵字列表
keywords = pd.read_excel("D:/UserData/Desktop/big data/國巨_news_各指標-2.xlsx",sheet_name = "關鍵字",encoding = "big5")
#看漲資料集
news_up = pd.read_excel("D:/UserData/Desktop/big data/國巨_news_up&down_5%.xlsx",sheet_name = "up",encoding = "big5")
#看跌資料集
news_down = pd.read_excel("D:/UserData/Desktop/big data/國巨_news_up&down_5%.xlsx",sheet_name = "down",encoding = "big5")

In [197]:
#建立關鍵字的向量空間(以前200個關鍵字為例)
words = []
for i in range(200):
    words.append(keywords["Collection"][i]) 

In [198]:
#在漲跟跌的的文章後分別加入"up" or "down" 的標籤
label_up = []
for i in range(len(news_up["content"])):
    label_up.append("up")
label_down = []
for i in range(len(news_down["content"])):
    label_down.append("down")

In [199]:
#新增label的欄位，後合併漲跟跌的資料集
datafram_up = pd.DataFrame({'date':news_up.date,'title':news_up.title,"content":news_up.content,"label":label_up})
datafram_down = pd.DataFrame({'date':news_down.date,'title':news_down.title,"content":news_down.content,"label":label_down})
datafram = pd.merge(datafram_up,datafram_down, on =["date",'title',"content","label"],how = "outer") 

In [200]:
#拆分成 訓練集8 : 測試集2 
datafram = datafram.sample(frac = 1)
train = datafram.iloc[:int(len(datafram.date)*0.8)]
test = datafram.iloc[int(len(datafram.date)*0.8):]

In [203]:
len(label_down)

1069

# Knn演算法＆文章向量化

In [186]:
#定義函式
#將文章向量化，每個維度是該關鍵字的TF值
def to_vector(x):
    import math
    vector_space = []
    for i in range(len(words)):
        if type(x)==float:
            continue
        if x.count(words[i]) !=0:
            log_value_TF = 1 + math.log10(x.count(words[i]))
            
        else:
            log_value_TF = 0
            
        vector_space.append(log_value_TF)
    return vector_space

#計算兩文章向量的餘弦相似度。
def cos (x,y):
    import math
    t = 0
    b_x = 0
    b_y = 0
    for i in range(len(x)):
        t += x[i]*y[i]
        b_x += x[i]**2
        b_y += y[i]**2
    if (math.sqrt(b_x)*math.sqrt(b_y)) == 0 :
        output = 0
    else:
        output = t/ (math.sqrt(b_x)*math.sqrt(b_y))
    return output

#將一測試文章丟入訓練集中，計算與餘弦相似度與之最接近的7筆文章
def Knn (x):
    output = ""
    t = to_vector(x)
    Knn_Rank = [0,0,0,0,0,0,0]
    text = [0,0,0,0,0,0,0]
    vote = ["0","0","0","0","0","0","0"]
    for i in range(len(train.content)):
        c = cos(to_vector(train.content.iloc[i]),t)
        for j in range(7):
            if c>Knn_Rank[j] and int(c)!=1  :
                for k in range(6,j,-1):
                    Knn_Rank[k] = Knn_Rank[k-1]
                    text[k] = text[k-1]
                    vote[j] = vote[k-1]
                Knn_Rank[j] = c
                text[j] = i
                vote[j] = train.label.iloc[i]
                break
    if vote.count("up") > vote.count("down"):
        ouput = "up"
    else:
        ouput = "down"
    return ouput

In [188]:
#依序將測試集的文章透過Knn演算法比對
P=[]
for i in range(len(test.content)):
    P.append(Knn(test.content.iloc[i]))
    

# 計算準確率

In [205]:

uu=0  #預測為up 真實為up
ud=0  #預測為up 真實為down
du=0  #預測為down 真實為up
dd=0  #預測為down 真實為down

for i in range(len(test.content)):
    if P[i] =="up" and test.label.iloc[i]=="up":
        uu+=1
    if P[i] =="down" and test.label.iloc[i]=="down":
        dd+=1
    if P[i] =="up" and test.label.iloc[i]=="down":
        ud+=1
    if P[i] =="down" and test.label.iloc[i]=="up":
        du+=1
accuracy=(uu+dd)/len(P)       # 正確率 accuracy：在所有情況中，正確判斷真假的比例。
precision=uu/(uu+ud)          # 精確率 precision：判斷為真的情況下，有多少是真的真。
recall=uu/(uu+du)             # 召回率 recall：為真的情況下，有多少被正確判斷出來。 
print("     真實為漲","真實為跌")
print("預測為漲 ",uu,"   ",ud)
print("預測為跌 ",du,"   ",dd)


print("\naccuracy = ",accuracy,'\nprecision = ',precision, "\nrecall = ",recall)

     真實為漲 真實為跌
預測為漲  90     78
預測為跌  137     120

accuracy =  0.49411764705882355 
precision =  0.5357142857142857 
recall =  0.3964757709251101
